In [98]:
%load_ext autoreload
%autoreload 2

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio
import seaborn as sns
import torch

from evolver import CrossoverType, MutationType, MatrixEvolver
from unet import UNet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
def read_tif_to_np(tif_path):
    """Reads a tif file and converts it into a numpy.ndarray.
    
    Arg:
        tif_path: The full path to the tif file to read.
    
    Returns:
        A numpy.ndarray containing the tif file data.
    
    """
    with rasterio.open(tif_path) as f:
        data = np.rollaxis(f.read(), 0, 3)
        return data
    
def get_tile_labels(base_dir, tile):
    """Retrieves the landcover labels data for a given tile.
    
    Args:
        base_dir: A directory to a tile block.
        tile: A tile name of format {block}/{block}{quad}_{quarter}_{garbage}.
    
    Returns:
        A ndarray containing the labels data for the tile.
    
    """
    dir_ext = "_lc.tif"
    tile_path = base_dir + tile + dir_ext
    return read_tif_to_np(tile_path)

def get_tile_features(base_dir, tile):
    """Retrieves the landcover features data for a given tile.
    
    Args:
        base_dir: A directory to a tile block.
        tile: A tile name of format {block}/{block}{quad}_{quarter}_{garbage}.
    
    Returns:
        A ndarray) containing the features data for the tile.
    
    """
    dir_ext = ".tif" 
    tile_path = base_dir + tile + dir_ext
    return read_tif_to_np(tile_path)

def apply_label_remap(labels, label_map):
    """Reassigns labels inplace in an numpy array given a provided mapping.
    
    Args:
        labels: An ndarray of labels.
        label_map: A dict[int, int] mapping label classes [original, new].
        
    """
    for l1, l2 in label_map.items():
        labels[labels == l1] = l2
        
def sample_patches(data, labels, patch_size, n_samples):
    """Generates image patches from a tile containing features and corresponding labels.
    
    Args:
        data: The x features of the image.
        labels: The y labels of the image.
        patch_size: An Iterable[int, int] size of the image patch to be extracted.
        n_samples: The number of samples to extract per tile.
        
    Returns:
        A list of x_patches and y_patches containg features and labels respectively.

    """
    height, width = patch_size
    x_patches = np.zeros(n_samples, height, wodt)
    y_patches = np.zeros()

    for i in range(n_samples):
        x = np.random.randint(0, data.shape[1] - width)
        y = np.random.randint(0, data.shape[0] - height)
        img = data[y:y + height, x:x + width, :].astype(np.float32)
        target = all_data_color_labels[count][y:y+height, x:x+width].copy()

    return x_patches, y_patches

In [59]:
# Define tiles to be used in testing, training and finetuning.
tile_dir_features = "/mnt/blobfuse/resampled-lc/data/v1/2015/states/md/md_1m_2015/39076/"
tile_dir_labels = "/mnt/blobfuse/esri-naip/v002/md/2015/md_100cm_2015/39076/" 
tiles = ["m_3907639_sw_18_1_20150815", "m_3907639_ne_18_1_20150815"]

In [60]:
# Read X and Y data for the tile and apply label transformations.
tile = tiles[0]
data = get_tile_features(tile_dir_features, tile)
labels = get_tile_labels(tile_dir_labels, tile)

water_forest_land_impervious_remap = {5: 4, 6: 4}
apply_label_remap(labels, water_forest_land_impervious_remap)

patch_size = (256, 256)
samples_per_tile = 5000
x, y = sample_patches(data, labels, patch_size, samples_per_tile)

In [100]:
model = UNet(in_channels = 256, n_classes = len(np.unique(y)), depth = 4)
layer_sizes = [[10, 10], [5, 5]]
dropout_mask_evolver = MatrixEvolver(layer_sizes, CrossoverType.UNIFORM, MutationType.FLIP_BIT)